In [ ]:
import numpy as np
import pandas as pd
import torch
import bioio

In [9]:
import struct
import tensorflow as tf
from tensorflow_datasets.core.features.features_dict import FeaturesDict
import bioio

import numpy as np
import pandas as pd
import torch
import bioio

# def numpy_to_torch(x):
#     if isinstance(x, str) or isinstance(x, bytes):
#         if isinstance(x, str):
#             x = x.encode('UTF-8')
#         x = np.frombuffer(x, dtype=np.uint8)
#     return torch.Tensor(x)

# def load_index(filepath):
#     return np.array(pd.read_csv(filepath, sep='\t', header=None)[0], dtype=np.int64)

class GFileTFRecord:
    def __init__(self, filepath, features=None, index=None):
        self._gfile_tfrecord = tf.io.gfile.GFile(filepath, 'rb')
        self.features = self._read_features(features) if features is not None else None
        self.index = self._read_index(index) if index is not None else None

    def __call__(self, offset, deserialize=True, to_numpy=False, to_torch=False, validate=False):
        try:
            proto = self._read_proto(offset, validate)
        except:
            raise ValueError(f'Invalid record at offset {offset}.')
        
        if (self.features is None) or (not deserialize):
            return proto
        else:
            return self.deserialize(proto, to_numpy, to_torch)
    
    def __getitem__(self, idx, **kwargs):
        if self.index is None:
            raise ValueError('Index not specified.')
        return self(self.index[idx], **kwargs)
    
    def __iter__(self):
        while True:
            try:
                proto = self._read_next_proto()
                if not proto:
                    break
                yield proto
            except:
                break
    
    def __len__(self):
        if self.index is not None:
            return len(self.index)
        else:
            if hasattr(self, '_len'):
                return self._len
            else:
                self._len = self._get_length()
                return self._len
    
    def _get_length(self):
        n = 0
        for _ in iter(self):
            n += 1
        return n

    @property
    def size(self):
        return self._gfile_tfrecord.size()

    def as_tf_dataset(self, cache=False, shuffle=False, deserialize=True):
        assert self.index is not None, 'Index not specified. Provide index or use `bioio.load_tfrecord` instead.'
        dataset = tf.data.Dataset.from_tensor_slices(self.index)
        dataset = dataset.map(self._read_proto_pyfunc)
        dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
        if deserialize:
            dataset = dataset.map(self.deserialize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
            dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
        if cache:
            dataset = dataset.cache()
        if shuffle:
            dataset = dataset.shuffle(len(self))
        dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
        return dataset
    
    def as_numpy_iterator(self, cache=False, shuffle=False):
        return self.as_tf_dataset(cache, shuffle, deserialize=True).as_numpy_iterator()

    def as_torch_iterator(self, cache=False, shuffle=False):
        for example in self.as_numpy_iterator(cache, shuffle):
            yield tf.nest.map_structure(bioio.torch.utils.numpy_to_torch, example)

    def deserialize(self, proto, to_numpy=False, to_torch=False):
        assert not (to_numpy and to_torch), 'Cannot convert to both numpy and torch.'

        if self.features is None:
            raise ValueError('Features not specified.')
        
        example = self.features.deserialize_example(proto)
        if to_numpy:
            example = tf.nest.map_structure(lambda x: x.numpy(), example)
        if to_torch:
            example = tf.nest.map_structure(lambda x: bioio.torch.utils.numpy_to_torch(x.numpy()), example)
        return example
    
    def _read_proto(self, offset, validate=False):
        # seek to offset
        self._gfile_tfrecord.seek(offset)
        return self._read_next_proto(validate)
    
    def _read_proto_pyfunc(self, offset):
        # proto_bytes = tf.py_function(lambda offset: self._read_proto(offset.numpy()), inp=[offset], Tout=tf.string)
        proto_bytes = tf.py_function(self._read_proto, inp=[offset], Tout=tf.string)
        proto_bytes.set_shape(shape=())
        return proto_bytes
    
    def _read_next_proto(self, validate=False):
        # get proto length
        proto_len_bytes = self._gfile_tfrecord.read(8)
        if len(proto_len_bytes) == 0:
            return None
        proto_len = struct.unpack('q', proto_len_bytes)[0]

        # proto length crc
        proto_len_crc = self._gfile_tfrecord.read(4)
        if validate:
            raise NotImplementedError('CRC validation not implemented.')

        # proto bytes
        proto_bytes = self._gfile_tfrecord.read(proto_len)

        # proto bytes crc
        proto_bytes_crc = self._gfile_tfrecord.read(4)
        if validate:
            raise NotImplementedError('CRC validation not implemented.')
        
        return proto_bytes

    def _read_features(self, features):
        if isinstance(features, FeaturesDict):
            return features
        elif isinstance(features, str):
            return bioio.tf.utils.features_from_json_file(features)
        else:
            raise ValueError(f'Invalid features type: {type(features)}')
        
    def _read_index(self, index):
        if isinstance(index, np.ndarray):
            return index
        elif isinstance(index, str):
            return bioio.tf.index.load_index(index)
        else:
            raise ValueError(f'Invalid features type: {type(index)}')


In [12]:
# gfile_tfrecord = GFileTFRecord('examples/windows.chr13.4.data.matrix.filtered.tfrecord', 'examples/windows.chr13.4.data.matrix.filtered.tfrecord.features.json', 'examples/windows.chr13.4.data.matrix.filtered.tfrecord.idx')
gfile_tfrecord = GFileTFRecord('examples/windows.chr13.4.data.matrix.filtered.tfrecord', 'examples/windows.chr13.4.data.matrix.filtered.tfrecord.features.json')

print(len(gfile_tfrecord))

# d = gfile_tfrecord.as_tf_dataset()
# print(d)
# # next(iter(d))
# # next(iter(gfile_tfrecord.as_numpy_iterator()))
next(iter(gfile_tfrecord.as_torch_iterator()))


1147


AssertionError: Index not specified. Provide index or use `bioio.load_tfrecord` instead.

In [ ]:
torch.tensor([1,2,3])

In [ ]:
# idx_dataset = iter(bioio.tf.index.load_index_to_dataset('examples/windows.chr13.4.data.matrix.filtered.tfrecord.idx'))
# print(next(idx_dataset))
# print(next(idx_dataset))


In [ ]:
gfile_tfrecord.index

In [ ]:
gfile_tfrecord[42]

In [ ]:
gfile_tfrecord.__getitem__(42, to_torch=True)

In [ ]:
import bioio
bioio.torch.utils

In [ ]:
from bioio.tf.ops import features_from_json_file
from bioio.tf.data import load_index

class GFileTFRecordsDataset(torch.utils.data.Dataset):
    def __init__(self, filepaths):
        self._gfile_tfrecords = [self._load_gfile_tfrecord(filepath) for filepath in filepaths] # [GFileTFRecord(filepath) for filepath in filepaths]
        self._gfile_tfrecords_lengths = [len(gfile_tfrecord) for gfile_tfrecord in self._gfile_tfrecords]
        self._idx_to_file_map = self._make_idx_to_file_map()

    def __getitem__(self, idx):
        file_idx, in_file_idx = self._idx_to_file_map[idx]
        return self._gfile_tfrecords[file_idx][in_file_idx]
    
    def __len__(self):
        return sum(self._gfile_tfrecords_lengths)

    def _load_gfile_tfrecord(self, filepath):
        features = features_from_json_file(filepath + '.features.json')
        index = load_index(filepath + '.idx')
        return GFileTFRecord(filepath, features, index)

    def _make_idx_to_file_map(self):
        idx_to_file_map = []
        for i, length in enumerate(self._gfile_tfrecords_lengths):
            for j in range(length):
                idx_to_file_map.append((i, j))
        return idx_to_file_map


In [ ]:
dataset = GFileTFRecordsDataset(['examples/windows.chr13.4.data.matrix.filtered.tfrecord', 'examples/windows.chr13.4.data.matrix.filtered.tfrecord'])
print(len(dataset))

In [ ]:
for _ in dataset:
    pass

In [ ]:
import bioio

tf_dataset = bioio.tf.load_indexed_tfrecord('examples/windows.chr13.4.data.matrix.filtered.tfrecord')
tf_dataset

In [ ]:
def load_index(filepath):
    return np.array(pd.read_csv(filepath, sep='\t', header=None)[0], dtype=np.int64)

load_index('examples/windows.chr13.4.data.matrix.filtered.tfrecord.idx')

In [ ]:
feat = bioio.tf.ops.features_from_json_file('examples/windows.chr13.4.data.matrix.filtered.tfrecord.features.json')

In [ ]:
len(gfile_tfrecord)

In [ ]:
next(iter(gfile_tfrecord))

In [ ]:
x = gfile_tfrecord(3401, deserialize=True, to_torch=True)
x

In [ ]:
type(x['meta'])

In [ ]:
torch.Tensor.byte(x['meta'], dtype=torch.uint8)

In [ ]:
torch.Tensor(int.from_bytes(b'abc', 'big'), dtype=torch.uint8)

In [ ]:
list(map(lambda x: int.from_bytes(x, 'big'), b'abc'))

In [ ]:
np.frombuffer(bytes(b'abc', encoding='UTF-8'), dtype=np.uint8)

In [ ]:
b'abc'

In [ ]:
import bioio

In [ ]:
dataset = bioio.tf.load_tfrecord('examples/windows.chr13.4.data.matrix.filtered.tfrecord', deserialize=False)
dataset = dataset.cache()
for _ in dataset:
    pass

In [ ]:
for _ in dataset:
    pass

In [ ]:
dataset_deser = bioio.tf.load_tfrecord('examples/windows.chr13.4.data.matrix.filtered.tfrecord', deserialize=True)
dataset_deser = dataset_deser.cache()
for _ in dataset_deser:
    pass

In [ ]:
for _ in dataset_deser:
    pass